In [1]:
# Import Libaries
import pandas as pd
import os
from sklearn.cluster import KMeans
import folium
import holidays

Create a Combined Dataframe

In [11]:
# Specify the file path
file_path_raw = 'Data/Raw Data/'

# Initialize the files list
files = []

# Define a dictionary to map month abbreviations to numeric values
month_dict = {
    'jan': 1,
    'feb': 2,
    'mar': 3,
    'apr': 4,
    'may': 5,
    'jun': 6,
    'jul': 7,
    'aug': 8,
    'sep': 9,
    'oct': 10,
    'nov': 11,
    'dec': 12
}

# Iterate through the files in the directory
for filename in os.listdir(file_path_raw):
    # Check if the current item is a file
    if os.path.isfile(os.path.join(file_path_raw, filename)):
        # Extract the month and year from the filename
        month = filename.split('-')[1][:3].lower()
        year = int(filename.split('-')[1][3:-4])

        # Combine the month and year into a sortable format
        sortable_date = f"{year:04d}-{month_dict[month]:02d}"

        # Append the filename and sortable date to the files list
        files.append((filename, sortable_date))

# Sort the files list based on the sortable date
files.sort(key=lambda x: x[1])


In [12]:
# Create an empty list to hold dataframes
df_list = []

# Loop through each file
for file, _ in files:
    # Read the CSV file into a dataframe and append it to the list
    df_list.append(pd.read_csv(os.path.join(file_path_raw, file)))

# Concatenate all dataframes in the list
df = pd.concat(df_list)

# Print the Data entries and columns
print(f'Number of trips: {df.shape[0]:,} \n\n{df.head()}')


Number of trips: 4,534,327 

          Date/Time      Lat      Lon    Base
0  4/1/2014 0:11:00  40.7690 -73.9549  B02512
1  4/1/2014 0:17:00  40.7267 -74.0345  B02512
2  4/1/2014 0:21:00  40.7316 -73.9873  B02512
3  4/1/2014 0:28:00  40.7588 -73.9776  B02512
4  4/1/2014 0:33:00  40.7594 -73.9722  B02512


Create new Date and Time Features

In [13]:
# Convert 'Date/Time' to datetime
df['Date/Time'] = pd.to_datetime(df['Date/Time'])

# Extract the date from date/time
df['Date'] = df['Date/Time'].dt.date

# Extract the time from date/time
df['Time'] = df['Date/Time'].dt.time

# Create new column with abbreviated month names
df['Month_Name'] = df['Date/Time'].dt.strftime('%b')  

# Create new column with month number
df['Month'] = df['Date/Time'].dt.month

# Create 'YearMonth' column
df['YearMonth'] = pd.to_datetime(df['Date']).dt.to_period('M')

# Extract the hour from 'Date/Time'
df['Hour'] = df['Date/Time'].dt.hour

# Extract the weekday from date
df['Weekday'] = pd.to_datetime(df['Date']).dt.day_name()

# Mapping the day_of_week values to 'Weekend' or 'Weekday'
df['Day_Type'] = df['Weekday'].map({'Monday': 'Weekday', 'Tuesday': 'Weekday', 'Wednesday': 'Weekday', 'Thursday': 'Weekday', 'Friday': 'Weekday', 'Saturday': 'Weekend', 'Sunday': 'Weekend'})

# Initialize the US holidays
us_holidays = holidays.US()

# Create a new column 'IsHoliday' and check if each date is a holiday
df['IsHoliday'] = df['Date'].apply(lambda x: x in us_holidays)

# Check the new columns
print(df.head())


            Date/Time      Lat      Lon    Base        Date      Time  \
0 2014-04-01 00:11:00  40.7690 -73.9549  B02512  2014-04-01  00:11:00   
1 2014-04-01 00:17:00  40.7267 -74.0345  B02512  2014-04-01  00:17:00   
2 2014-04-01 00:21:00  40.7316 -73.9873  B02512  2014-04-01  00:21:00   
3 2014-04-01 00:28:00  40.7588 -73.9776  B02512  2014-04-01  00:28:00   
4 2014-04-01 00:33:00  40.7594 -73.9722  B02512  2014-04-01  00:33:00   

  Month_Name  Month YearMonth  Hour  Weekday Day_Type  IsHoliday  
0        Apr      4   2014-04     0  Tuesday  Weekday      False  
1        Apr      4   2014-04     0  Tuesday  Weekday      False  
2        Apr      4   2014-04     0  Tuesday  Weekday      False  
3        Apr      4   2014-04     0  Tuesday  Weekday      False  
4        Apr      4   2014-04     0  Tuesday  Weekday      False  


In [14]:
# Function to categorize time of day
def categorize_time(time):
    if time >= pd.Timestamp('05:00:00').time() and time < pd.Timestamp('12:00:00').time():
        return 'morning'
    elif time >= pd.Timestamp('12:00:00').time() and time < pd.Timestamp('17:00:00').time():
        return 'afternoon'
    elif time >= pd.Timestamp('17:00:00').time() and time < pd.Timestamp('21:00:00').time():
        return 'evening'
    else:
        return 'night'

# Apply the function to the 'Time' column
df['Time_of_day'] = df['Time'].apply(categorize_time)

print(df.head())

            Date/Time      Lat      Lon    Base        Date      Time  \
0 2014-04-01 00:11:00  40.7690 -73.9549  B02512  2014-04-01  00:11:00   
1 2014-04-01 00:17:00  40.7267 -74.0345  B02512  2014-04-01  00:17:00   
2 2014-04-01 00:21:00  40.7316 -73.9873  B02512  2014-04-01  00:21:00   
3 2014-04-01 00:28:00  40.7588 -73.9776  B02512  2014-04-01  00:28:00   
4 2014-04-01 00:33:00  40.7594 -73.9722  B02512  2014-04-01  00:33:00   

  Month_Name  Month YearMonth  Hour  Weekday Day_Type  IsHoliday Time_of_day  
0        Apr      4   2014-04     0  Tuesday  Weekday      False       night  
1        Apr      4   2014-04     0  Tuesday  Weekday      False       night  
2        Apr      4   2014-04     0  Tuesday  Weekday      False       night  
3        Apr      4   2014-04     0  Tuesday  Weekday      False       night  
4        Apr      4   2014-04     0  Tuesday  Weekday      False       night  


Create Cluster and Cluster features

In [15]:
# Store the latitudinal and longitudinal coordinates
X = df[['Lat', 'Lon']]

# Create a KMeans instance with 10 clusters
kmeans = KMeans(n_clusters=10, random_state=0)

# Fit the model to the data
kmeans.fit(X)

# Get the cluster assignments for each data point
df['Cluster'] = kmeans.labels_

# Print the first few rows of the DataFrame to see the new 'Cluster' column
print(df.head())

            Date/Time      Lat      Lon    Base        Date      Time  \
0 2014-04-01 00:11:00  40.7690 -73.9549  B02512  2014-04-01  00:11:00   
1 2014-04-01 00:17:00  40.7267 -74.0345  B02512  2014-04-01  00:17:00   
2 2014-04-01 00:21:00  40.7316 -73.9873  B02512  2014-04-01  00:21:00   
3 2014-04-01 00:28:00  40.7588 -73.9776  B02512  2014-04-01  00:28:00   
4 2014-04-01 00:33:00  40.7594 -73.9722  B02512  2014-04-01  00:33:00   

  Month_Name  Month YearMonth  Hour  Weekday Day_Type  IsHoliday Time_of_day  \
0        Apr      4   2014-04     0  Tuesday  Weekday      False       night   
1        Apr      4   2014-04     0  Tuesday  Weekday      False       night   
2        Apr      4   2014-04     0  Tuesday  Weekday      False       night   
3        Apr      4   2014-04     0  Tuesday  Weekday      False       night   
4        Apr      4   2014-04     0  Tuesday  Weekday      False       night   

   Cluster  
0        1  
1        7  
2        7  
3        4  
4        4  


In [16]:
# Calculate centroids
centroids = kmeans.cluster_centers_

# Create a map centered around the average coordinates
map_centroids = folium.Map(location=[centroids[:, 0].mean(), centroids[:, 1].mean()], zoom_start=2)

# Add markers for each centroid
for i, centroid in enumerate(centroids):
    folium.Marker(location=[centroid[0], centroid[1]], 
                  popup=f'Cluster {i}',  # Display cluster number on click
                  icon=folium.Icon(icon="cloud")  # Create an Icon
                 ).add_to(map_centroids)

# Display the map
map_centroids

In [17]:
# Create a dictionary that maps cluster numbers to location names
cluster_names = {
    7: 'Lower Manhattan',
    4: 'Midtown',
    1: 'Central Park',
    3: 'Williamsburg',
    0: 'Brooklyn',
    9: 'John F. Kennedy Airport',
    2: 'Levittown',
    5: 'Bronx',
    8: 'Newark Liberty Airport',
    6: 'LaGuardia Airport'
}

# Create a new column with the location names
df['Location'] = df['Cluster'].map(cluster_names)
print(df.head())

            Date/Time      Lat      Lon    Base        Date      Time  \
0 2014-04-01 00:11:00  40.7690 -73.9549  B02512  2014-04-01  00:11:00   
1 2014-04-01 00:17:00  40.7267 -74.0345  B02512  2014-04-01  00:17:00   
2 2014-04-01 00:21:00  40.7316 -73.9873  B02512  2014-04-01  00:21:00   
3 2014-04-01 00:28:00  40.7588 -73.9776  B02512  2014-04-01  00:28:00   
4 2014-04-01 00:33:00  40.7594 -73.9722  B02512  2014-04-01  00:33:00   

  Month_Name  Month YearMonth  Hour  Weekday Day_Type  IsHoliday Time_of_day  \
0        Apr      4   2014-04     0  Tuesday  Weekday      False       night   
1        Apr      4   2014-04     0  Tuesday  Weekday      False       night   
2        Apr      4   2014-04     0  Tuesday  Weekday      False       night   
3        Apr      4   2014-04     0  Tuesday  Weekday      False       night   
4        Apr      4   2014-04     0  Tuesday  Weekday      False       night   

   Cluster         Location  
0        1     Central Park  
1        7  Lower Ma

In [18]:
# Define the file path
file_path = 'Data/Processed Data/'

# Save the csv file
df.to_csv(file_path + 'preprocessed_data.csv', index=False)

Creating Dataset for Model Training

In [19]:
# Load the dataframe
df = pd.read_csv(file_path + 'preprocessed_data.csv')

In [20]:
# Group the dataframe by 'Cluster', 'Date', and 'Hour' and calculate the average
grouped_df = df.groupby(['Cluster', 'Date']).sum()

# Add a column for the number of data entries
grouped_df['Pickups'] = df.groupby(['Cluster', 'Date']).size()

# Reset the index to add the columns used for grouping
grouped_df = grouped_df.reset_index()

# Create a list of columns to be removed
columns_to_remove = ['Lat', 'Lon', 'Month', 'IsHoliday', 'Hour']

# Remove the columns from the dataframe
grouped_df = grouped_df.drop(columns=columns_to_remove)

# Calculate the average pickups per hour
grouped_df['Pickups Per Hour'] = grouped_df['Pickups'] / 24

# Round the number to two decimals
decimal_places = 2
grouped_df['Pickups Per Hour'] = grouped_df['Pickups Per Hour'].round(decimal_places)

# Convert the Date column to datetime if it's not already in that format
grouped_df['Date'] = pd.to_datetime(grouped_df['Date'])

print(grouped_df)

      Cluster       Date  Pickups  Pickups Per Hour
0           0 2014-04-01      762             31.75
1           0 2014-04-02      808             33.67
2           0 2014-04-03      869             36.21
3           0 2014-04-04     1211             50.46
4           0 2014-04-05     1590             66.25
...       ...        ...      ...               ...
1825        9 2014-09-26      838             34.92
1826        9 2014-09-27      728             30.33
1827        9 2014-09-28     1374             57.25
1828        9 2014-09-29     1130             47.08
1829        9 2014-09-30      807             33.62

[1830 rows x 4 columns]


In [21]:
file_path_cluster = 'Data/Model Data/'

for cluster_number in range(10):
    # Filter the data for the selected cluster
    cluster_data = grouped_df[grouped_df['Cluster'] == cluster_number]

    # Save the csv file
    cluster_data.to_csv(file_path_cluster + 'cluster_' + str(cluster_number) + '_data.csv', index=False)